In [2]:
pip install biopython

     |████████████████████████████████| 2.3 MB 5.0 MB/s 


In [87]:
import pandas as pd
import numpy as np
import re
import regex
from itertools import permutations, combinations_with_replacement, product

In [2]:
path = r"/content/drive/MyDrive/Colab Notebooks/Py GenDS/dna2.fasta"
seqs = {}
with open(path) as f:
  for line in f:
    if line[0]==">":
      curr_seq = line.split()[0].split("|")[-1]
      seqs[curr_seq]=""
    else:
      seqs[curr_seq] = seqs[curr_seq] + line[:-1]

In [211]:
len(seqs.keys())

18

In [3]:
lens = pd.DataFrame([[key, len(value)] for key, value in zip(seqs.keys(),seqs.values())], columns = ['id', 'len']).set_index('id').sort_values(by='len', ascending = False)
print(lens.head(3))
print("")
print(lens.tail(3))

      len
id       
255  4894
16   4804
91   4635

     len
id      
88   890
322  442
346  115


In [4]:
def reading_frame(seq, frame):
  out_seq = ""
  i = 0
  offset = (frame+2)%3
  for k in range(offset):
    out_seq = out_seq + seq[i]
    i += 1
  for n in range(len(seq) - offset):
    if ((i - offset)%3==0)&(i>0):
      out_seq = out_seq + " "
    out_seq = out_seq + seq[i]
    i += 1
  return out_seq


def undo_frame(seq):
  out_seq = seq.replace(" ", "")
  return out_seq

In [45]:
new_seq = reading_frame(seqs['527'],3)

In [58]:
long_seq = undo_frame(re.findall('(ATG.*?(?:TAA|TGA|TAG))', new_seq)[2])
re.search(long_seq, undo_frame(new_seq)).start()

635

In [64]:
orfs = pd.DataFrame([[key, max([*map(len, map(undo_frame, re.findall('(ATG.*?(?:TAA|TGA|TAG))', reading_frame(value, 3))))], default=0)] for key, value in zip(seqs.keys(),seqs.values())], columns = ['id', 'orf_len']).set_index('id').sort_values(by='orf_len', ascending = False)
print(orfs.head(3))
print("")
print(orfs.tail(3))

     orf_len
id          
527     1821
16      1644
454     1401

     orf_len
id          
584      132
88         0
346        0


In [95]:
def repeats(seq, n):
  reps = {}
  for rep in [''.join(comb) for comb in product('ATGC', repeat=n)]:
    reps[rep]=""
  for key in reps.keys():
    reps[key] = len(regex.findall("(?:"+key+")+?", seq, overlapped=True))
  return reps

In [97]:
def repeats_all(seqs, n):
  reps = {}
  for rep in [''.join(comb) for comb in product('ATGC', repeat=n)]:
      reps[rep]=0
  for seq in seqs.values():
    for key in reps.keys():
      reps[key] = reps[key] + len(regex.findall("(?:"+key+")+?", seq, overlapped=True))
  return reps

In [103]:
def repeats_all_v2(seqs, n):
  reps = {}
  for seq in seqs.values():
    for base in range(len(seq)-n):
      rep = seq[base:base + n]
      try:
        reps[rep] += 1
      except:
        reps[rep] = 1
  return reps

In [109]:
rep_dict = repeats_all_v2(seqs, 7)

In [110]:
repis = pd.DataFrame([[key, value] for key, value in zip(rep_dict.keys(),rep_dict.values())], columns = ['rep', 'count']).set_index('rep').sort_values(by='count', ascending = False)
print(repis.head(10))
print("")
print(repis.tail(3))

         count
rep           
CGCGCCG     63
CGCCGCG     62
GCCGCGC     61
GCGCGCG     59
GCGCGGC     58
CGCGACG     57
CGCGGCG     57
CGCGCGC     56
GCCGCCG     52
CGGCGCG     51

         count
rep           
AGTACAA      1
CTTGTAC      1
ACGCATA      1
